In [1]:
import os

In [2]:
username = os.environ['JUPYTERHUB_USER']
hiveaddr = os.environ['HIVE_SERVER_2']
print("Operating as: {0}".format(username))

Operating as: mezghani


In [3]:
from pyhive import hive

conn = hive.connect(host=hiveaddr, 
                    port=10000,
                    username=username) 
cur = conn.cursor()